In [8]:
import sys
import numpy as np
import math

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector

from qiskit.extensions import Initialize

In [9]:
Pigeon  = "ATGGTGCTGTCTGCCAACGACAAGAGCAACGTGAAGGCCGTCTTCGGCAAAATCGGCGGCCAGGCCGGTGACTTGGGTGGTGAAGCCCTGGAGAGGTTGTTCATCACCTACCCCCAGACCAAGACCTACTTCCCCCACTTCGACCTGTCACATGGCTCCGCTCAGATCAAGGGGCACGGCAAGAAGGTGGCGGAGGCACTGGTTGAGGCTGCCAACCACATCGATGACATCGCTGGTGCCCTCTCCAAGCTGAGCGACCTCCACGCCCAAAAGCTCCGTGTGGACCCCGTCAACTTCAAACTGCTGGGTCACTGCTTCCTGGTGGTCGTGGCCGTCCACTTCCCCTCTCTCCTGACCCCGGAGGTCCATGCTTCCCTGGACAAGTTCGTGTGTGCCGTGGGCACCGTCCTTACTGCCAAGTACCGTTAA"
Duck    = "ATGGTGCTGTCTGCGGCTGACAAGACCAACGTCAAGGGTGTCTTCTCCAAAATCGGTGGCCATGCTGAGGAGTATGGCGCCGAGACCCTGGAGAGGATGTTCATCGCCTACCCCCAGACCAAGACCTACTTCCCCCACTTTGACCTGCAGCACGGCTCTGCTCAGATCAAGGCCCATGGCAAGAAGGTGGCGGCTGCCCTAGTTGAAGCTGTCAACCACATCGATGACATTGCGGGTGCTCTCTCCAAGCTCAGTGACCTCCACGCCCAAAAGCTCCGTGTGGACCCTGTCAACTTCAAATTCCTGGGCCACTGCTTCCTGGTGGTGGTTGCCATCCACCACCCCGCTGCCCTGACCCCAGAGGTCCACGCTTCCCTGGACAAGTTCATGTGCGCCGTGGGTGCTGTGCTGACTGCCAAGTACCGTTAG"
Chicken = "ATGGTGCTGTCCGCTGCTGACAAGAACAACGTCAAGGGCATCTTCACCAAAATCGCCGGCCATGCTGAGGAGTATGGCGCCGAGACCCTGGAAAGGATGTTCACCACCTACCCCCCAACCAAGACCTACTTCCCCCACTTCGATCTGTCACACGGCTCCGCTCAGATCAAGGGGCACGGCAAGAAGGTAGTGGCTGCCTTGATCGAGGCTGCCAACCACATTGATGACATCGCCGGCACCCTCTCCAAGCTCAGCGACCTCCATGCCCACAAGCTCCGCGTGGACCCTGTCAACTTCAAACTCCTGGGCCAATGCTTCCTGGTGGTGGTGGCCATCCACCACCCTGCTGCCCTGACCCCGGAGGTCCATGCTTCCCTGGACAAGTTCTTGTGCGCCGTGGGCACTGTGCTGACCGCCAAGTACCGTTAA"

In [10]:
def encode_bitstring(bitstring, qr, cr, inverse=False):
    """
    create a circuit for constructing the quantum superposition of the bitstring
    """
    n = math.ceil(math.log2(len(bitstring))) + 2            #number of qubits
    assert n > 2, "the length of bitstring must be at least 2"
    
    qc = QuantumCircuit(qr, cr)
    
    #the probability amplitude of the desired state
    desired_vector = np.array([ 0.0 for i in range(2**n) ]) #initialize to zero

################################################################################################################    
 #         Cambio requerido en el código para corregir error presentado   #
    
    qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inicialización
    inverse_qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inversión de la inicialización

################################################################################################################

    amplitude = np.sqrt(1.0/(2**(n-2)-83))
    
    for i, b in enumerate(bitstring):
        pos = i * 4
        if b == "01" or b == "C":
            pos += 1
        if b == "10" or b == "G":
            pos += 2
        if b == "11" or b == "T":
            pos += 3
        desired_vector[pos] = amplitude

#####################################a###########################################################################    
 #         Cambio requerido en el código para corregir error presentado   #
    
    qc_init = Initialize(desired_vector)
    
 ################################################################################################################       
    
    if not inverse:
#         qc.initialize(desired_vector, qr)

        qc.append(qc_init, qr)  #  Cambio requerido en el código para corregir error presentado   #
    
        qc.barrier(qr)
        
    else:
#         qc.initialize(desired_vector, qr).inverse()  #invert the circuit

        inverse_qc_init = qc_init.gates_to_uncompute()  # Cambio requerido en el código para corregir error presentado   #
        qc.append(inverse_qc_init, qr)  # Cambio requerido en el código para corregir error presentado   #
        
        qc.barrier(qr)
        for i in range(n):
            qc.measure(qr[i], cr[i])
    print()
    return qc

In [11]:
n = math.ceil(math.log2(len(Pigeon))) + 2    #number of qubits
qr = QuantumRegister(n)
cr = ClassicalRegister(n)

qc_Pigeon     = encode_bitstring(Pigeon, qr, cr)
qc_Duck = encode_bitstring(Duck, qr, cr)
qc_Chicken = encode_bitstring(Chicken, qr, cr)

circs = {"Pigeon": qc_Pigeon, "Duck": qc_Duck, "Chicken": qc_Chicken}

In [12]:
inverse_qc_Pigeon     = encode_bitstring(Pigeon,     qr, cr, inverse=True)
inverse_qc_Duck = encode_bitstring(Duck, qr, cr, inverse=True)
inverse_qc_Chicken = encode_bitstring(Chicken, qr, cr, inverse=True)

inverse_circs = {"Pigeon": inverse_qc_Pigeon, "Duck": inverse_qc_Duck, "Chicken": inverse_qc_Chicken}

In [14]:
from qiskit import IBMQ, BasicAer

key = "Duck"       #the name of the code used as key to find similar ones
key1 = "Pigeon"

# use local simulator
backend = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inverse_circs:
    if other_key == key:
        continue
        
    combined_circs[other_key] = circs[key] + inverse_circs[other_key]   #combined circuits to look for similar codes
    job = execute(combined_circs[other_key], backend=backend,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0
    
    print("Similarity score of",key,"and",other_key,"is",sim_score)
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score

    if other_key == key1:
        continue        
    combined_circs[other_key] = circs[key1] + inverse_circs[other_key]   #combined circuits to look for similar codes
    job = execute(combined_circs[other_key], backend=backend,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0 
    print("Similarity score of",key1,"and",other_key,"is",sim_score)
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score
        
        
print("[ANSWER]", key,"is most similar to", most_similar)
print("[ANSWER]", key1,"is most similar to", most_similar)

Similarity score of Duck and Pigeon is 0.7081298828125
Similarity score of Duck and Chicken is 0.7774658203125
Similarity score of Pigeon and Chicken is 0.7225341796875
[ANSWER] Duck is most similar to Chicken
[ANSWER] Pigeon is most similar to Chicken
